# SAA_top测试
## 1. 加载Overlay

In [45]:
import numpy as np
from pynq import allocate
import random
import time
import saa_top_driver
from saa_insn_driver import * 
from saa_utils import * 
# 创建 SaaDriver 实例
saa_driver = saa_top_driver.SaaDriver("saa_top.bit")

saa_top Overlay downloaded successfully!


## 3.连续缓存申请

In [59]:
# 定义指令缓冲区大小
insn_count = 4000 # 最多能容纳2000条指令
block_size = 1*MATRIX_WIDTH # 以脉动阵列大小作为分块
# 定义buffer大小,这是执行一个批量的大小
row =  5*MATRIX_WIDTH
col =  5*MATRIX_WIDTH
col1 = 5*MATRIX_WIDTH

# 定义PS端缓冲区,不使用cache，数据类型注意
# instruct_buffer = allocate(shape = (insn_count), cacheable = 0, dtype = Instruct_DataType)
input_buffer = allocate(shape = (row*col), cacheable = 0, dtype = Input_DataType)
weight_buffer = allocate(shape = (col*col1), cacheable = 0, dtype = Weight_DataType)
bias_buffer = allocate(shape = (row*col1), cacheable = 0, dtype = Output_DataType)
output_buffer  = allocate(shape = (row,col1), cacheable = 0, dtype = Output_DataType)

## 4.测试数据生成

In [60]:
# 随机生成矩阵并存储到相应的数据缓冲区中
np.random.seed(2)  # 设置随机种子以确保生成的随机数相同
input_matrix = np.random.randint(0, 100, size=(row, col), dtype=Input_DataType)
weight_matrix = np.random.randint(0, 100, size=(col, col1), dtype=Weight_DataType)
bias_matrix = np.random.randint(0, 100, size=(row, col1), dtype=Output_DataType)
print("Randomly generated input buffer:")
print(input_matrix)
print("\nRandomly generated weight buffer:")
print(weight_matrix)
print("\nRandomly generated bias buffer:")
print(bias_matrix)


# 执行打包操作
pack_matrix_to_buffer(input_matrix, MATRIX_WIDTH, input_buffer)
pack_matrix_to_buffer(weight_matrix, MATRIX_WIDTH, weight_buffer)
pack_matrix_to_buffer(bias_matrix, MATRIX_WIDTH, bias_buffer)
print("Packed input buffer:")
print(input_buffer)
print("Packed input buffer:")
print(weight_buffer)
print("Packed bias buffer:")
print(bias_buffer)

# 将输入矩阵转换为np.int32类型，以避免溢出
input_matrix_int32 = input_matrix.astype(np.int32)
weight_matrix_int32 = weight_matrix.astype(np.int32)
# 定义input_buffer和weight_buffer的矩阵乘法结果的结果矩阵
pt0 = time.perf_counter()
result_matrix = np.dot(input_matrix_int32, weight_matrix_int32)+bias_matrix
pt1 = time.perf_counter()
time_sw = pt1 - pt0
print("pure software: %fs" % time_sw)
# 打印矩阵乘法结果
print("Matrix bias multiplication result:")
print(result_matrix)

Randomly generated input buffer:
[[40 92 29 15 10 97 47 25 35  6 72 22 46 54 12 43 30 82 73 75]
 [ 4 24 57 29  7 45 14 82 34 82 16 84 49  1  8 39 95 90 99 52]
 [75 42 50 85 10  8 30 47 20 53 30 63 43 54 76 31 52 90 74 78]
 [68 20 25 33 32  2 31 78 81 37 91  7 39 74 90 46 67 84 34 52]
 [33  4 91  3 42 83 56 22 55 47 51 80 65 56 57 11 73 38 44 66]
 [31 30 90 33 62 84 11 58 78  6 83 69 64 56 88 67 93 78 45 69]
 [37 99 20 88 84 57 68 78 87  1 46  9 76 70 29 54 16 95 69 49]
 [73 83 25 88 62 29 31 64 54 24 66 41  5 46 80 14 13 32 52 70]
 [ 9 76 68 24 76 85 16 50 91 57  4 48 74 56 90 74 96 27 63 77]
 [68 79 56 49 74 19 39 55 14 89 42 46 64  8 26 18 94 50 37 51]
 [49 35 15 39 42  7 75  8 49 87  1 62 41 17 11 99 22  3 28 82]
 [43 75 40 39 20 73 14 22 30 57 12 41 87  8 38  1 51 22 47 90]
 [90 81 62 73  2 83 12  0  0 96  0 31 55 43 22 55 99 32 51 70]
 [26 87 85 55 80  8 91 48 17 76 86 60 66 38 10 28 38 79 40 43]
 [78 34 10 90 60 21 76 70  9  6 68 90 29  9 55 48 70 70 40 86]
 [92 86 76 70 57 81 11

## 分块矩阵乘法

In [61]:
# 执行分块矩阵乘法
blocked_gemm_test(saa_driver,
              row, 
              col1, 
              col, 
              input_buffer, 
              weight_buffer,
              bias_buffer, 
              output_buffer, 
              block_size, 
              1)
# 检查输出
print(output_buffer)
output_buffer[:]=0
# del output_buffer

INFO - Blocked GEMM test: dim_I=20, dim_J=20, dim_K=20, block=4, bias_use=1
compute_insn_count: 126
insn_size: 227
insn_idx: 227
done: 0
INFO - Saa run time: 0.000314s
INFO - Synchronization time: 0.313733ms
INFO - Throughput: 0.076498GOPs/s
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0

In [43]:
# 随机生成矩阵并存储到相应的数据缓冲区中
np.random.seed(4)  # 设置随机种子以确保生成的随机数相同
input_matrix = np.random.randint(0, 100, size=(row, col), dtype=Input_DataType)
weight_matrix = np.random.randint(0, 100, size=(col, col1), dtype=Weight_DataType)
bias_matrix = np.random.randint(0, 100, size=(row, col1), dtype=Output_DataType)
print("Randomly generated input buffer:")
print(input_matrix)
print("\nRandomly generated weight buffer:")
print(weight_matrix)
print("\nRandomly generated bias buffer:")
print(bias_matrix)


# 执行打包操作
pack_matrix_to_buffer(input_matrix, MATRIX_WIDTH, input_buffer)
pack_matrix_to_buffer(weight_matrix, MATRIX_WIDTH, weight_buffer)
pack_matrix_to_buffer(bias_matrix, MATRIX_WIDTH, bias_buffer)
print("Packed input buffer:")
print(input_buffer)
print("Packed input buffer:")
print(weight_buffer)
print("Packed bias buffer:")
print(bias_buffer)

# 将输入矩阵转换为np.int32类型，以避免溢出
input_matrix_int32 = input_matrix.astype(np.int32)
weight_matrix_int32 = weight_matrix.astype(np.int32)
# 定义input_buffer和weight_buffer的矩阵乘法结果的结果矩阵
pt0 = time.perf_counter()
result_matrix = np.dot(input_matrix_int32, weight_matrix_int32)+bias_matrix
pt1 = time.perf_counter()
time_sw = pt1 - pt0
print("pure software: %fs" % time_sw)
# 打印矩阵乘法结果
print("Matrix bias multiplication result:")
print(result_matrix)

Randomly generated input buffer:
[[68 15 46 32 15 55 23 12 69 66 53 44  1 87  1  9]
 [91 87 46 54 72 65 27 50 91 30 50  9 24 58 36 81]
 [55 54 36 31 56 57 94 78 24  1 55 43 75 50 55 67]
 [64 57 53 54 78 36 94  9 34 50 35  5 71 44 28 82]
 [ 1 38 27 50  5 18 31 92 52  6 11 92  3 37 64  0]
 [40 26 55 16 41 21 74 74 21 96 24 73 78 91 79 38]
 [76  2 78 28 85 98 56 32 55 68 66 18 78 11 46 16]
 [87 36 53 30 68 33 79  8 78 39 49 78 94  4 66 45]
 [58 11 47 10 87 89 75 73 32 69 40 34 69 93 67 42]
 [17 84 44 45 55 33 84 47 19 32 87 82 16 70 74 25]
 [61  5 45 28 57  0 43  3 42 93 77 62 59 87 79 70]
 [15 98 86 86 95  8 37  7 57 76 94 48  4 50 98 83]
 [85 17 96 13  5 16 79 66 81 15 50 27 89 39  9 73]
 [84  1 40 59 48 16 58 48 18 11 76 15 54 23 88 63]
 [ 6 52 67 46 94 58 35 22 80 67 72 62 52 52 61 62]
 [32 89 38 75 37 24 96 23 61 27 56 68 33 56 31 70]]

Randomly generated weight buffer:
[[38 68 50  5 35 59 81 45 71 18 54 19 73 59 37 96]
 [82 73  7 90 55  1 38  0 53 62 73 47 28 88 91 85]
 [38 64 94 17

In [44]:
# 执行分块矩阵乘法
blocked_gemm_test(saa_driver,
              row, 
              col1, 
              col, 
              input_buffer, 
              weight_buffer,
              bias_buffer, 
              output_buffer, 
              block_size, 
              1)
# 检查输出
print(output_buffer)

INFO - Blocked GEMM test: dim_I=16, dim_J=16, dim_K=16, block=4, bias_use=1
compute_insn_count: 65
insn_size: 130
insn_idx: 130
done: 0
INFO - Saa run time: 0.000459s
INFO - Synchronization time: 0.459225ms
INFO - Throughput: 0.026758GOPs/s
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


## 7. 回收缓冲区

In [5]:
# 查看完成后清空缓冲区
del output_buffer